In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns

## Load the dataset

In [2]:
df = pd.read_csv('../data/raw/srcsc-2025-dam-data-for-students.csv')

## Rename features

In [3]:
def general_rename(str):
    return str.lower().replace(' ', '_')

In [4]:
renamed_df = df.rename(columns=general_rename).rename(columns={
    'height_(m)': 'height',
    'length_(km)': 'length',
    'volume_(m3)': 'volume',
    'surface_(km2)': 'surface',
    'drainage_(km2)': 'drainage',
    'distance_to_nearest_city_(km)': 'distance_to_nearest_city',
    'loss_given_failure_-_prop_(qm)': 'dam_repair_loss',
    'loss_given_failure_-_liab_(qm)': 'damage_loss',
    'loss_given_failure_-_bi_(qm)': 'business_interruption_loss'
})

## Feature Engineering

#### years_completed

In [5]:
# calculate the dam age
renamed_df['age'] = 2025 - renamed_df['year_completed']

#### years_modified

In [6]:
#calculate the length of given list
def list_len(list):
    if list == 0:
        return 0
    return len(set(list))

modification_pattern = r'(\d{4})'
# count the number of unique modifications
renamed_df['modification_count'] = renamed_df['years_modified'].str.findall(pat=modification_pattern).fillna(0).apply(list_len)

In [7]:
# convert all elements in the list to integer and calcualte the difference between 2025 and max value in the list
def years_from_cal(list):
    if list == -1:
        return list
    list = [int(i) for i in list]
    list.sort()
    return 2025 - list[-1]

# calculate the years taken from last modification
renamed_df['years_from_modification'] = renamed_df['years_modified'].str.findall(modification_pattern).fillna(-1).apply(years_from_cal)
# fill null value with dam age
renamed_df['years_from_modification'] = renamed_df['years_from_modification'].where(cond=renamed_df['years_from_modification'] != -1, other=renamed_df['age'])


#### last_inspection_date

In [8]:
# Calculate the years taken from last inspection
renamed_df['years_from_inspection'] = renamed_df['last_inspection_date'].str.findall(r'\d{2}/\d{2}/(\d{4})').fillna(-1).apply(years_from_cal)
# fill null value with the dam age
renamed_df['years_from_inspection'] = renamed_df['years_from_inspection'].where(cond=renamed_df['years_from_inspection'] != -1, other=renamed_df['age'])


#### inspection_frequency

In [9]:
# fill na with mode category: 0
renamed_df['inspection_frequency'] = renamed_df['inspection_frequency'].fillna(0)
renamed_df['inspection_frequency'] = renamed_df['inspection_frequency'].apply(lambda x: 1/x if x != 0 else 0)

#### assessment_date

In [10]:
# Calculate the years taken from last inspection
renamed_df['years_from_assessment'] = renamed_df['assessment_date'].str.findall(r'\d{2}/\d{2}/(\d{4})').fillna(-1).apply(years_from_cal)
# fill null value with the dam age
renamed_df['years_from_assessment'] = renamed_df['years_from_assessment'].where(cond=renamed_df['years_from_assessment'] != -1, other=renamed_df['age'])


In [11]:
# drop used features
engineered_df = renamed_df.drop(columns=['years_modified', 'last_inspection_date', 'year_completed', 'assessment_date'])

## Impute outliers

In [12]:
# impute outliers with median of outlier-removed dataframe
'''
def impute_outliers(df, feature):
    # calculate IQR
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    # calculate median without outliers
    removed_df = df[(df[feature] >= lower) & (df[feature] <= upper)]
    new_median = removed_df[feature].median()
    # impute outliers with median calculated
    df.loc[(df[feature] < lower) | (df[feature] > upper), feature] = new_median
    return
'''

'\ndef impute_outliers(df, feature):\n    # calculate IQR\n    Q1 = df[feature].quantile(0.25)\n    Q3 = df[feature].quantile(0.75)\n    IQR = Q3 - Q1\n    lower = Q1 - 1.5*IQR\n    upper = Q3 + 1.5*IQR\n    # calculate median without outliers\n    removed_df = df[(df[feature] >= lower) & (df[feature] <= upper)]\n    new_median = removed_df[feature].median()\n    # impute outliers with median calculated\n    df.loc[(df[feature] < lower) | (df[feature] > upper), feature] = new_median\n    return\n'

In [13]:
for feature in ['length', 'volume', 'height', 'surface', 'drainage']:
    engineered_df[feature] = engineered_df[feature].replace(0, np.nan)
'''
# impute outliers with median
for feature in ['length', 'volume', 'surface', 'drainage']:
    impute_outliers(engineered_df, feature)
outliers_imputed_df = engineered_df
'''

"\n# impute outliers with median\nfor feature in ['length', 'volume', 'surface', 'drainage']:\n    impute_outliers(engineered_df, feature)\noutliers_imputed_df = engineered_df\n"

In [14]:
outliers_imputed_df = engineered_df

## Handle Missing Data

In [15]:
for feature in outliers_imputed_df.columns:
    if outliers_imputed_df[feature].isna().sum() > 0:
        print(f'{feature} : {outliers_imputed_df[feature].isna().sum()}  {round(outliers_imputed_df[feature].isna().sum()/len(outliers_imputed_df)*100, 2)}%')


primary_purpose : 1184  5.69%
primary_type : 257  1.24%
height : 24  0.12%
length : 2925  14.06%
volume : 12838  61.7%
surface : 4668  22.44%
drainage : 7671  36.87%
spillway : 12786  61.45%
distance_to_nearest_city : 10229  49.16%
assessment : 2537  12.19%
dam_repair_loss : 7  0.03%
damage_loss : 12  0.06%
business_interruption_loss : 10730  51.57%
age : 1384  6.65%
years_from_modification : 1284  6.17%
years_from_inspection : 659  3.17%
years_from_assessment : 785  3.77%


#### numerical features

In [16]:
for feature in ['height', 'length', 'volume', 'surface', 'drainage', 'distance_to_nearest_city', 'age', 'years_from_modification', 'years_from_inspection', 'years_from_assessment']:
    outliers_imputed_df[feature] = outliers_imputed_df[feature].fillna(outliers_imputed_df[feature].median())

#### categorical_feature

In [17]:
# fill null with NA
for feature in ['primary_purpose', 'spillway']:
    outliers_imputed_df[feature] = outliers_imputed_df[feature].fillna('No Rocord')

# fill null with mode category
for feature in ['primary_type', 'assessment']:
    outliers_imputed_df[feature] = outliers_imputed_df[feature].fillna(outliers_imputed_df[feature].mode()[0])

## Save dataset

In [18]:
outliers_imputed_df.to_csv('../data/curated/visulisation_df.csv', index=False)